In [1]:
from tqdm import tqdm_notebook as tqdm
import json
import datetime
import re
import os
import json
import requests
import math
import collections
import pandas as pd
import numpy as np
import scipy.sparse as sp

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
base_dir = 'kp_data'
sample_subdir = 'sample'

user2vote_csr_file = os.path.join(base_dir, sample_subdir, "user2vote.npz")
users_file         = os.path.join(base_dir, sample_subdir, "users.csv")
movies_file        = os.path.join(base_dir, sample_subdir, "movies.csv")

## Load sample

In [3]:
def load_int2int_dict(filename):
    with open(os.path.join(base_dir, sample_subdir, filename), 'r') as f:
        j = json.load(f)
        return {int(i): int(j[i]) for i in j}

def load_sample():
    user2vote_csr = sp.load_npz(user2vote_csr_file)
    users  = pd.read_csv(users_file, index_col=0)
    movies = pd.read_csv(movies_file, index_col=0)
    movie_id2id = load_int2int_dict('movie_id2id.json')
    id2movie_id = load_int2int_dict('id2movie_id.json')
    id2user_id = load_int2int_dict('id2user_id.json')
    user_id2id = load_int2int_dict('user_id2id.json')
    
    assert (max(id2movie_id.keys()) == len(id2movie_id) - 1)
    assert (len(movie_id2id) == len(id2movie_id))
    print ("len(id2movie_id)", len(id2movie_id))
           
    assert (max(id2user_id.keys()) == len(id2user_id) - 1)
    assert (len(user_id2id) == len(id2user_id))
    print ("len(id2user_id)", len(id2user_id))
           
    print ('user2vote_csr.shape', user2vote_csr.shape)
    
    return user2vote_csr, users, movies, movie_id2id, id2movie_id, user_id2id, id2user_id

In [4]:
%%time 

user2vote_csr, users, movies, movie_id2id, id2movie_id, user_id2id, id2user_id = load_sample()

len(id2movie_id) 13776
len(id2user_id) 218007
user2vote_csr.shape (218007, 13776)
Wall time: 3 s


In [5]:
# movies[movies['id']==4873]

In [6]:
# id2movie_id[0] == 12252

In [7]:
# movie_id2id[4873] == 1

In [8]:
movies

,id,href,name_rus,name_eng,rating,rating_count,critics_rating,imdb_rating,imdb_rating_count,year,duration,genres,countries
5,12252,/film/skiny-1992-12252/,Скины,Romper Stomper,7.036,16346.0,79,6.8,34385,1992,94,"криминал,боевик,драма",Австралия
6,4873,/film/4873/,Фрэнки и Джонни,Frankie and Johnny,7.562,5658.0,78,6.7,24874,1991,118,"драма,мелодрама,комедия",США
7,678269,/film/678269/,Космическая станция 76,Space Station 76,4.963,1258.0,67,4.9,6742,2014,95,"фантастика,драма,комедия",США
9,4743,/film/4743/,Три мушкетера,The Three Musketeers,6.253,4066.0,31,6.4,44526,1993,105,"боевик,приключения,семейный,комедия,мелодрама","Великобритания,США,Австрия"
11,161180,/film/pevets-2006-161180/,Певец,El cantante,6.635,1052.0,25,5.5,4297,2006,116,"драма,музыка,биография",США
15,86964,/film/ad-2005-86964/,Ад,L'enfer,6.942,866.0,67,6.9,2216,2005,102,драма,"Италия,Франция,Бельгия,Япония"
17,477855,/film/477855/,Зона 51,Area 51,4.224,1211.0,17,4.2,10661,2015,95,"фантастика,ужасы,триллер",США
20,974817,/film/974817/,Стена,The Wall,6.258,3180.0,66,6.2,18654,2017,88,"триллер,драма,военный",США
21,277328,/film/varkraft-2016-277328/,Варкрафт,Warcraft,7.599,138208.0,28,6.9,217267,2016,123,"боевик,приключения,фэнтези","Китай,США,Канада,Япония"
24,596231,/film/596231/,Страховщик,Autómata,5.793,20433.0,29,6.1,47148,2014,110,"фантастика,боевик,триллер","Болгария,США,Канада,Испания"


In [9]:
user2vote_csr

<218007x13776 sparse matrix of type '<class 'numpy.float64'>'
	with 33582215 stored elements in Compressed Sparse Row format>

### Recommendation algorithms got from https://geekbrains.ru/events/657

#### Recommendation functions

In [29]:
def get_id2value(row):
#     if not row.has_sorted_indices:
#         row = row.sorted_indices()

    return {int(indice): int(row.data[i]) 
            for i, indice in enumerate(row.indices)}

def sim_pearson (prefs_csr, person1, person2, 
                            person1_i2v = None, 
                            person2_i2v = None,
                            count_thres = 1,
                            votes_thres = 0.8):
    if person1_i2v is None:
        person1_i2v = get_id2value(prefs_csr[person1])
    if person2_i2v is None:
        person2_i2v = get_id2value(prefs_csr[person2])

    si = person1_i2v.keys() & person2_i2v.keys()
    
    if len(si) < count_thres or len(si) < len(person1_i2v.keys())*votes_thres :
        return 0.0

    corr = np.corrcoef([person1_i2v[i] for i in si],
                       [person2_i2v[i] for i in si])[0,1]

    if math.isnan(corr):
        return 0.0

    return corr

def top_matches(prefs_csr, person, n = 5, similarity = sim_pearson):
    scores = []
    found_absoulete_correlation = 0
    for other, _ in enumerate(prefs_csr):
                              
        if other == person:
            continue
                              
        rate = similarity(prefs_csr, person, other)
        if rate == 1.0:
            found_absoulete_correlation += 1
        
        scores.append((rate, other))
        
        if found_absoulete_correlation == n:
            break

    scores.sort()
    scores.reverse()
    return scores[:n]

def get_reccomentations(prefs_csr, person = -1, similarity = sim_pearson, 
                        n = 5, person_mid2vote = None, votes_count_thres = 10):
    totals    = {} # {mid: sum [sim(i) * rate(i)]}
    sim_sums  = {} # {mid: sum [sim(i)] }
    counter   = collections.Counter()
    for other, _ in enumerate(prefs_csr):
        if other == person: continue
            
        person1_i2v = get_id2value(prefs_csr[person]) if person_mid2vote is None else person_mid2vote
        person2_i2v = get_id2value(prefs_csr[other])
        sim = similarity(prefs_csr, person, other, person1_i2v, person2_i2v)
        
        if sim <= 0.1: continue 

        for item in person2_i2v.keys():
            if item not in person1_i2v.keys(): # did not watched yet
                totals.setdefault(item, 0.0)
                totals[item] += sim * person2_i2v[item]
                
                sim_sums.setdefault(item, 0.0)
                sim_sums[item] += sim 
                
                counter[item] += 1

    rank = [(totals[item]/sim_sums[item], item)
             for item in totals.keys()
             if item in sim_sums.keys() and counter[item] > votes_count_thres]
        
    rank.sort()
    rank.reverse()
    return rank[:n]

def get_movie_descr_by_id(mid, fields = ('name_rus', 'name_eng', 'year', 'rating', 'genres', 'countries')):
    m = movies.loc[movies['id'] == id2movie_id[mid]]
    return [m[f].tolist()[0] for f in fields]
def get_user_descr_by_id(uid, fields = ('user_id', 'nickname', 'gender', 'country', 'city', 'birth_date')):
    u = users.loc[users['user_id'] == id2user_id[uid]]
    return [u[f].tolist()[0] for f in fields]

def get_filmid2vote(votes):
    _filmid2vote = {
        int(href.split('/')[-2].split('-')[-1]): votes[href]['vote']
        for href in votes
    }

    _filmid2vote = {movie_id2id[movie_id]: vote 
                    for movie_id, vote in _filmid2vote.items()
                    if movie_id in movie_id2id}
    return _filmid2vote

In [11]:
# movies.loc[(movies['name_rus'] == 'Профессионал') & (movies['year'] == 2018)]['id']

In [12]:
# get_movie_descr_by_id(movie_id2id[1038843])

## Recommend!
## Работает достаточно долго!

In [13]:
from kp_utils import *

# real users
user_1 = 15666129
user_2 = 1979843
user_3 = 15425314

# establishing session
s = requests.Session()
s.headers.update({
        'Referer': 'http://www.kinopoisk.ru',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36',
        'Host': "www.kinopoisk.ru",
        'Upgrade-Insecure-Requests': '1',
        'Cookie': "mobile=no; noflash=true; hideBlocks=0; my_perpages=%7B%2277%22%3A200%2C%2279votes%22%3A200%7D",
        'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,de;q=0.6',
        'Referer' : 'https://www.kinopoisk.ru',
        'Accept'  : "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        'Connection': 'keep-alive'    
    })

In [14]:
%%time
votes_1 = get_user_votes_for_films(user_i=user_1, session=s)

Wall time: 1.41 s


In [15]:
%%time
votes_2 = get_user_votes_for_films(user_i=user_2, session=s)

Wall time: 785 ms


In [16]:
%%time
votes_3 = get_user_votes_for_films(user_i=user_3, session=s)

Wall time: 2.01 s


In [31]:
for mid, vote in get_filmid2vote(votes_1).items():
    print (vote, get_movie_descr_by_id(mid))

9 ['Жизнь Пи', 'Life of Pi', 2012, 7.822999999999999, 'драма,приключения,фэнтези', 'Великобритания,Канада,Тайвань,США']
9 ['Эффект бабочки', 'The Butterfly Effect', 2003, 8.169, 'фантастика,триллер', 'США,Канада']
10 ['Прислуга', 'The Help', 2011, 8.162, 'драма', 'ОАЭ,Индия,США']
8 ['Шрек', 'Shrek', 2001, 7.978, 'семейный,фэнтези,приключения,мультфильм,комедия', 'США']
8 ['Назад в будущее\xa03', 'Back to the Future Part III', 1990, 8.169, 'приключения,фантастика,вестерн,комедия', 'США']
8 ['Терминал', 'The Terminal', 2004, 8.068, 'драма,мелодрама,комедия', 'США']
10 ['Перл-Харбор', 'Pearl Harbor', 2001, 7.923999999999999, 'история,военный,боевик,драма,мелодрама', 'США']
7 ['В поисках Немо', 'Finding Nemo', 2003, 7.862999999999999, 'семейный,фэнтези,приключения,мультфильм,комедия', 'США']
8 ['Великий Гэтсби', 'The Great Gatsby', 2013, 7.94, 'драма,мелодрама', 'Австралия,США']
9 ['Бесславные ублюдки', 'Inglourious Basterds', 2009, 7.867000000000001, 'боевик,драма,военный,комедия', 'Герма

In [32]:
for mid, vote in get_filmid2vote(votes_2).items():
    print (vote, get_movie_descr_by_id(mid))

4 ['Девушка в поезде', 'The Girl on the Train', 2016, 6.541, 'драма,детектив,криминал,триллер', 'США']
9 ['Фантастические твари и где они обитают', 'Fantastic Beasts and Where to Find Them', 2016, 7.56, 'приключения,семейный,фэнтези', 'Великобритания,США']
8 ['Как приручить дракона\xa02', 'How to Train Your Dragon\xa02', 2014, 7.923999999999999, 'семейный,фэнтези,приключения,мультфильм,комедия', 'США']
8 ['Город героев', 'Big Hero\xa06', 2014, 7.9289999999999985, 'семейный,боевик,фантастика,мультфильм,комедия', 'США']
5 ['Интерстеллар', 'Interstellar', 2014, 8.586, 'драма,приключения,фантастика', 'Великобритания,США']
8 ['Одержимость', 'Whiplash', 2013, 8.324, 'драма,музыка', 'США']
10 ['Мальчик в полосатой пижаме', 'The Boy in the Striped Pyjamas', 2008, 8.174, 'драма,военный', 'Великобритания,США']


In [33]:
for mid, vote in get_filmid2vote(votes_3).items():
    print (vote, get_movie_descr_by_id(mid))

9 ['Ла-Ла Ленд', 'La La Land', 2016, 7.904, 'музыка,драма,мелодрама,мюзикл,комедия', 'Гонконг,США']
8 ['Безумный Макс: Дорога ярости', 'Mad Max: Fury Road', 2015, 7.814, 'боевик,приключения,фантастика', 'Австралия,США']
8 ['Тачки\xa03', 'Cars\xa03', 2017, 7.102, 'семейный,спорт,приключения,мультфильм,комедия', 'США']
7 ['Стартрек: Бесконечность', 'Star Trek: Beyond', 2016, 6.857, 'боевик,приключения,фантастика,триллер', 'Гонконг,США,Китай']
9 ['Темный рыцарь', 'The Dark Knight', 2008, 8.505, 'триллер,боевик,драма,фантастика,криминал', 'Великобритания,США']
8 ['Титаник', 'Titanic', 1997, 8.369, 'драма,мелодрама', 'США']
8 ['Мстители: Война бесконечности', 'Avengers: Infinity War', 2018, 7.892, 'боевик,приключения,фантастика,фэнтези', 'США']
8 ['Чёрная Пантера', 'Black Panther', 2018, 6.54, 'боевик,приключения,фантастика', 'Корея Южная,ЮАР,Австралия,США']
9 ['Аватар', 'Avatar', 2009, 7.944, 'боевик,драма,приключения,фантастика', 'Великобритания,США']
7 ['Звёздные войны: Пробуждение силы'

In [34]:
%%time

for rate, mid in get_reccomentations(user2vote_csr, n = 20, person_mid2vote=get_filmid2vote(votes_1), votes_count_thres = 100):
    print (round(rate, 2), get_movie_descr_by_id(mid))

9.31 ['Форрест Гамп', 'Forrest Gump', 1994, 8.916, 'драма,мелодрама', 'США']
8.8 ['Игры разума', 'A Beautiful Mind', 2001, 8.558, 'драма,биография,мелодрама', 'США']
8.8 ['Гладиатор', 'Gladiator', 2000, 8.587, 'боевик,драма,приключения', 'Великобритания,США']
8.78 ['Жизнь прекрасна', 'La vita è bella', 1997, 8.628, 'драма,военный,комедия', 'Италия']
8.76 ['Крестный отец', 'The Godfather', 1972, 8.735, 'драма,криминал', 'США']
8.73 ['Криминальное чтиво', 'Pulp Fiction', 1994, 8.62, 'криминал,триллер,комедия', 'США']
8.67 ['Темный рыцарь', 'The Dark Knight', 2008, 8.505, 'триллер,боевик,драма,фантастика,криминал', 'Великобритания,США']
8.64 ['ВАЛЛ·И', 'WALL·E', 2008, 8.302999999999999, 'приключения,мультфильм,фантастика,семейный', 'США']
8.63 ['Американская история\xa0X', 'American History\xa0X', 1998, 8.298, 'драма,криминал', 'США']
8.61 ['Остров проклятых', 'Shutter Island', 2009, 8.492, 'драма,детектив,триллер', 'США']
8.59 ['Семь', 'Se7en', 1995, 8.296, 'драма,детектив,криминал,трилл

In [35]:
%%time

for rate, mid in get_reccomentations(user2vote_csr, n = 20, person_mid2vote=get_filmid2vote(votes_2), votes_count_thres = 100):
    print (round(rate, 2), get_movie_descr_by_id(mid))

9.38 ['Побег из Шоушенка', 'The Shawshank Redemption', 1994, 9.112, 'драма', 'США']
9.34 ['Зеленая миля', 'The Green Mile', 1999, 9.062, 'драма,детектив,криминал,фэнтези', 'США']
9.21 ['Форрест Гамп', 'Forrest Gump', 1994, 8.916, 'драма,мелодрама', 'США']
9.13 ['Список Шиндлера', "Schindler's List", 1993, 8.818999999999997, 'драма,биография,история', 'США']
9.05 ['Король Лев', 'The Lion King', 1994, 8.773, 'семейный,драма,мюзикл,приключения,мультфильм', 'США']
9.01 ['1+1', 'Intouchables', 2011, 8.809, 'драма,биография,комедия', 'Франция']
8.95 ['Леон', 'Léon', 1994, 8.682, 'драма,криминал,триллер', 'Франция']
8.92 ['Начало', 'Inception', 2010, 8.665, 'боевик,драма,фантастика,детектив,триллер', 'Великобритания,США']
8.91 ['Властелин колец: Возвращение Короля', 'The Lord of the Rings: The Return of the King', 2003, 8.615, 'драма,приключения,фэнтези', 'Новая Зеландия,США']
8.88 ['Назад в будущее', 'Back to the Future', 1985, 8.625, 'приключения,фантастика,комедия', 'США']
8.84 ['ВАЛЛ·И', 

In [36]:
%%time

for rate, mid in get_reccomentations(user2vote_csr, n = 20, person_mid2vote=get_filmid2vote(votes_3), votes_count_thres = 100):
    print (round(rate, 2), get_movie_descr_by_id(mid))

9.26 ['Форрест Гамп', 'Forrest Gump', 1994, 8.916, 'драма,мелодрама', 'США']
9.16 ['Начало', 'Inception', 2010, 8.665, 'боевик,драма,фантастика,детектив,триллер', 'Великобритания,США']
9.05 ['Список Шиндлера', "Schindler's List", 1993, 8.818999999999997, 'драма,биография,история', 'США']
9.05 ['Властелин колец: Возвращение Короля', 'The Lord of the Rings: The Return of the King', 2003, 8.615, 'драма,приключения,фэнтези', 'Новая Зеландия,США']
8.99 ['Матрица', 'The Matrix', 1999, 8.491, 'боевик,фантастика', 'США']
8.96 ['Бойцовский клуб', 'Fight Club', 1999, 8.65, 'драма,криминал,триллер', 'Германия,США']
8.93 ['Назад в будущее', 'Back to the Future', 1985, 8.625, 'приключения,фантастика,комедия', 'США']
8.93 ['Терминатор 2: Судный день', 'Terminator 2: Judgment Day', 1991, 8.306000000000001, 'боевик,фантастика,триллер', 'Франция,США']
8.92 ['Леон', 'Léon', 1994, 8.682, 'драма,криминал,триллер', 'Франция']
8.91 ['Король Лев', 'The Lion King', 1994, 8.773, 'семейный,драма,мюзикл,приключе